In [57]:
import features
import api
import pandas as pd
from api import get_note_by_name

In [52]:
index = api.index

query_note = get_note_by_name("2021-11-24")
feature_df = features.base_features(query_note, index)

In [58]:
def get_items_jaccard_long(feature_df: pd.DataFrame, n_items: int = 5):
    result_df = feature_df
    result_df = result_df[result_df["is_daily"] == False]
    result_df = result_df[result_df["name_n_words"] > 2]
    result_df = result_df[result_df["plaintext_n_words"] >= 10]
    result_df = result_df[result_df["distance"] >= 2]
    result_df = result_df[result_df["jaccard"] > 0.0]
    result_df = result_df.sort_values("jaccard", ascending=False).head(n_items)

    return result_df

def get_items_jaccard_short(feature_df: pd.DataFrame, n_items: int = 5):
    result_df = feature_df
    result_df = result_df[result_df["is_daily"] == False]
    result_df = result_df[result_df["name_n_words"] <= 2]
    result_df = result_df[result_df["plaintext_n_words"] >= 10]
    result_df = result_df[result_df["distance"] >= 2]
    result_df = result_df[result_df["jaccard"] > 0.0]
    result_df = result_df.sort_values("jaccard", ascending=False).head(n_items)

    return result_df

ranked_df = get_items_jaccard_short(feature_df, 50).reset_index(drop=True)

def pair_jaccard(name1, name2):
    for n1, n2, jaccard in index.jaccard_pairs:
        if n1 == name1 and n2 == name2 or n1 == name2 and n2 == name1:
            #print(f"{jaccard:.3f} - {name1} <-> {name2}")
            return jaccard
    
    #print(f"NONE - {name1} <-> {name2}")
    return 0.0

PENALTY_FACTOR = 0.5
penalties = []
for i, row in ranked_df.iterrows():
    #print(f"RANK {i}: {row['name']}")
    if i == 0:
        continue
    previous_rows = ranked_df.iloc[0:i]
    
    penalty = 0.0
    for j, prevrow in previous_rows.iterrows():
        prev_jaccard = pair_jaccard(row["name"], prevrow["name"])
        #print(f"{prev_jaccard}, {i}")
        penalty -= PENALTY_FACTOR * prev_jaccard / i # divide by i so that maximum possible penalty is constant across rank
    
    penalties.append(penalty)
    
ranked_df["penalty"] = pd.Series(penalties)
ranked_df["adjusted_jaccard"] = ranked_df["jaccard"] + ranked_df["penalty"]

print(ranked_df.head(10)["name"])
print(ranked_df.sort_values("adjusted_jaccard", ascending=False).head(10)["name"])

0      Mimetic desire
1            System 2
2               Prune
3         Celebristan
4              Babble
5       Freshmanistan
6       Vulnerability
7          Sam Harris
8         Writing MOC
9    Creating content
Name: name, dtype: object
0      Mimetic desire
1            System 2
3         Celebristan
6       Vulnerability
4              Babble
2               Prune
5       Freshmanistan
7          Sam Harris
8         Writing MOC
9    Creating content
Name: name, dtype: object
